# Fonctions utiles

### Import

In [16]:
import pandas as pd
import numpy as np
import os
import glob

### Conversion mathématique

In [34]:
def kN_t(kN=0, t_kg=0, unite_sortie='t'):
    '''
    Convertisseur kN <-> t, kg
        param: kN - kiloNewton
               t_kg - tonne ou kilogramme
               unite_sortie - Unité attendue en sortie de calcul : kN, t ou kg
        return: (float) Résultat du calcul de convertion
    '''
    
    if unite_sortie == 'kN':
        return t_kg/0.102
    elif unite_sortie == 't':
        return kN*0.102
    elif unite_sortie == 'kg':
        return kN*102
    else:
        print("L'unité n'est pas reconnue.\nChoix possible : \n\t- kN (kiloNewtons)\n\t- t (tonnes)\n\t- kg (kilogrammes)")
        return 0

In [35]:
def ms_kmh(ms=0, kmh=0, unite_sortie='kmh'):
    '''
    Convertisseur m/s <-> km/h
        param: ms - Mètre / seconde
               kmh - Kilomètre / heure
               unite_sortie - Unité attendue en sortie de calcul : ms ou kmh
        return: (float) Résultat du calcul de convertion
    '''
    
    if unite_sortie == 'kmh':
        return ms*3.6
    elif unite_sortie == 'ms':
        return kmh/3.6
    else:
        print("L'unité n'est pas reconnue.\nChoix possible : \n\t- kmh (km/h)\n\t- ms (m/s)")
        return 0

In [36]:
def hex_to_dec(element):
        new_list = []
        for elt in element:
            new_list.append(int(elt, 16))
            
        return new_list

### Découpage Series et conversion DataFrame

In [2]:
'''
Découpage des 'Warning Flags' et conversion en int
    param: WF_series - Warning Flags Series
    return: (DataFrame - int64) 8 colonnes contenant les warning flags par groupe de bits
'''
def warningFlags_dummies(WF_series):
    def normalisation_warningFlags(element):
        limit = 8 # Fixed by norm .nswd files
        len_elt = len(element)
        add_list = []
        for i in range(limit):
            add_list.append('0')

        if len_elt <= limit:
            alpha = limit-len_elt
            element = add_list[:alpha] + element

        return element
    
    normalisationWarningFlags = WF_series.apply(lambda x: normalisation_warningFlags(hex_to_dec(list(x))))
    warnings = pd.DataFrame.from_items(zip(normalisationWarningFlags.index, normalisationWarningFlags.values),\
                                       columns=['W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1', 'W0'], orient='index')
    
    return warnings.astype('int64')

In [40]:
'''
Découpage du 'timestamp' et conversion en int
    param: TS_series - Timestamp Series
    return: (Dataframe - int64) 7 colonnes contenant respectivement une donnée du timestamp: Year, Month, Day, ...
'''
def timestamp_dummies(TS_series):
    splitTimestamp = TS_series.str.split('-')
    time = pd.DataFrame.from_items(zip(splitTimestamp.index, splitTimestamp.values),\
                                   columns=['Year', 'Month', 'Day', 'Hour', 'Minute', 'Seconde', 'Millisecondes'],\
                                   orient='index')
    
    return time.astype('int64')

In [38]:
'''
Découpage de 'axle groups' et conversion en int
    param: AG_series - Axle Groups Series
    return: (Dataframe - int64) x colonnes dépendant du nombre d'axes totals de AG_series contenant respectivement un
            roupe d'axe chacune
'''
def axleGroups_dummies(AG_series):
    limit = np.max(AG_series.apply(lambda x: len(x)))
    
    def normalisation_axleGroups(element, limit):
        len_elt = len(element)
        add_list = []
        for i in range(limit):
            add_list.append('0')

        if len_elt <= limit:
            alpha = limit-len_elt
            element += add_list[:alpha]

        return element
    
    normalisationAxleGroups = AG_series.apply(lambda x: normalisation_axleGroups(hex_to_dec(list(x)), limit))

    # Création colonnes axle groups
    columns = []
    base_name = 'AG_'
    for i in range(1,limit+1):
        columns.append(base_name + str(i))
    
    axle = pd.DataFrame.from_items(zip(normalisationAxleGroups.index, normalisationAxleGroups.values),\
                                       columns=columns, orient='index')
    
    return axle.astype('int64')

### Recherche sur DataFrame ou Serie

In [5]:
'''
Recherche éléments uniques dans pandas Series
    param: Series - Pandas Series où rechercher les éléments uniques
    return: (DataFrame) Ensemble des éléments uniques du pandas Series
'''
def unique_element(Series):
    element = []
    for val in Series:
        if val not in element:
            element.append(val)
            
    return pd.DataFrame(element, columns=[Series.name])

In [6]:
'''
Recherche éléments uniques dans pandas dataframe
    param: DataFrame - Pandas DataFrame où rechercher les éléments uniques
    return: (DataFrame) Ensemble des éléments uniques de la pandas DataFrame
'''
def unique_element_df(DataFrame):
    df = pd.DataFrame()
    for col in DataFrame:
        df = pd.concat([df, unique_element(DataFrame[col])], axis=1)

    return df

In [1]:
def count_null(DataFrame):
    '''
    Comptage des élements NaN par colonne dans DataFrame
        param: DataFrame - DataFrame où l'on cherche à compter les NaN 
        return: (DataFrame) Listant l'ensemble des colonnes en comptant leur nombre de NaN
    '''
    return pd.DataFrame(data={'Missing':DataFrame.isnull().sum()}).sort_values(['Missing'])

### Liste redondante

In [8]:
'''
Création liste charge sur essieux et distance entre essieux
    param: bare_str_var - String contenant le préfix de la variable avant le chiffre incrémenté
           var_from - Int de départ de l'incrémentation de la liste
           var_to - Int de fin de la liste
    return: (list) Liste de l'ensemble des variables incrémentés commençant par 'bare_str_var'
'''
def list_multiple_var(bare_str_var, var_from=1, var_to=1):
    list_return = []
    
    try:
        if var_from > var_to:
            raise ValueError
        
        for i in range(var_from, var_to+1):
            list_return.append(bare_str_var+'{}'.format(i))
            
        return list_return
        
    except ValueError:
        print("'var_from' doit être inférieur ou égale à 'var_to'")

### Fichiers système

In [9]:
'''
Récupérer chemins des fichiers avec 'extension'
    param: source - String de chemin parent ou commencer à chercher les fichiers
           extension - String de l'extension à rechercher
    return: (list) Tous les chemins des fichiers avec 'extension'
'''
def find_all_extension(source, extension):
    list_ext = []
    
    for elt in glob.glob(source):
        if os.path.isdir(elt):
            list_ext.extend(find_all_extension(elt + '\\*', extension))
        else:
            if extension in elt:
                list_ext.append(elt)
          
    return list_ext